In [1]:
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# get data files
TRAIN_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/train-data.tsv"
TEST_DATA_URL = "https://raw.githubusercontent.com/beaucarnes/fcc_python_curriculum/master/sms/valid-data.tsv"

train_file_path = tf.keras.utils.get_file("train-data.tsv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("valid-data.tsv", TEST_DATA_URL)

131072/118774 [=================================] - 0s 0us/step


In [3]:
# convert data to pandas dataframe and change labels to numbers
df_train = pd.read_csv(train_file_path,sep='\t',header=None)
df_test = pd.read_csv(test_file_path,sep='\t',header=None)
df_train[0] = df_train[0].replace("ham", 0)
df_train[0] = df_train[0].replace("spam", 1)
df_test[0] = df_test[0].replace("ham", 0)
df_test[0] = df_test[0].replace("spam", 1)
df_test[0]=df_test[0].astype('int64')
df_train[0]=df_train[0].astype('int64')
# convert data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((df_train[1], df_train[0]))
test_dataset = tf.data.Dataset.from_tensor_slices((df_test[1], df_test[0]))
# build vocab list


import tensorflow_datasets as tfds

tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
for text_tensor, _ in train_dataset.concatenate(test_dataset):
  some_tokens = tokenizer.tokenize(text_tensor.numpy())
  vocabulary_set.update(some_tokens)

vocab_size = len(vocabulary_set)
vocab_size

8741

In [4]:
# create encoder based on vocab list
encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set)
# encode datasets
def encode(text_tensor, label):
  encoded_text = encoder.encode(text_tensor.numpy())
  return encoded_text, label

def encode_map_fn(text, label):
  encoded_text, label = tf.py_function(encode, 
                                       inp=[text, label], 
                                       Tout=(tf.int64, tf.int64))
  encoded_text.set_shape([None])
  label.set_shape([])

  return encoded_text, label


train_dataset_encoded = train_dataset.map(encode_map_fn)
test_dataset_encoded = test_dataset.map(encode_map_fn)
# check what data looks like after encoding
for train_example, train_label in train_dataset_encoded.take(2):
  print('Encoded text:', train_example[:10].numpy())
  print('Label:', train_label.numpy())

Encoded text: [6730 1400  915 3469  871 2345 2298 1027  234 4908]
Label: 0
Encoded text: [1380 7050  611 4494 7366]
Label: 0


In [5]:
# prepare data for training, padded_batch is used to make all reviews the same length while batching
BUFFER_SIZE = 1000

train_batches = (
    train_dataset_encoded
    .shuffle(BUFFER_SIZE)
    .padded_batch(32))

test_batches = (
    test_dataset_encoded
    .padded_batch(32))
# build neural network model
model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, 16),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(1, activation='sigmoid')])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          139888    
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 139,905
Trainable params: 139,905
Non-trainable params: 0
_________________________________________________________________


In [6]:
# train model
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=test_batches,
                    validation_steps=30)

Epoch 1/10
131/131 [==============================] - 3s 16ms/step - loss: 0.5667 - accuracy: 0.8464 - val_loss: 0.4397 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 2s 14ms/step - loss: 0.3604 - accuracy: 0.8679 - val_loss: 0.3091 - val_accuracy: 0.8646
Epoch 3/10
131/131 [==============================] - 2s 14ms/step - loss: 0.2740 - accuracy: 0.8768 - val_loss: 0.2638 - val_accuracy: 0.8771
Epoch 4/10
131/131 [==============================] - 2s 14ms/step - loss: 0.2427 - accuracy: 0.8928 - val_loss: 0.2335 - val_accuracy: 0.8979
Epoch 5/10
131/131 [==============================] - 2s 14ms/step - loss: 0.2119 - accuracy: 0.9117 - val_loss: 0.2067 - val_accuracy: 0.9125
Epoch 6/10
131/131 [==============================] - 2s 14ms/step - loss: 0.1837 - accuracy: 0.9289 - val_loss: 0.1823 - val_accuracy: 0.9240
Epoch 7/10
131/131 [==============================] - 2s 14ms/step - loss: 0.1578 - accuracy: 0.9407 - val_loss: 0.1617 - val_accuracy: 0.9365

In [8]:
# evaluate model
loss, accuracy = model.evaluate(test_batches)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

44/44 [==============================] - 1s 12ms/step - loss: 0.1198 - accuracy: 0.9547
Loss:  0.11983327567577362
Accuracy:  0.954741358757019


In [9]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  encoded_pred_text = encoder.encode(pred_text)
  encoded_pred_text = tf.cast(encoded_pred_text, tf.float32)
  prediction = model.predict(tf.expand_dims(encoded_pred_text, tf.constant(0))).tolist()
  prediction = prediction[0]
  if prediction[0] < .5:
    prediction.append("ham")
  else:
    prediction.append("spam")

  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.006971180438995361, 'ham']


In [10]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()

You passed the challenge. Great job!


# New Section